In [8]:
import pandas as pd
import numpy as np


from sklearn.cluster import KMeans, DBSCAN
# from sklearn.neighbors import NearestNeighbors

# removing outliers
from scipy import stats

import seaborn as sns
import matplotlib.pyplot as plt
# from matplotlib import cm
# from mpl_toolkits.mplot3d import axes3d, Axes3D, proj3d
# from mpl_toolkits.mplot3d.art3d import Poly3DCollection


#### Display options

%matplotlib inline
# %pylab inline
# %matplotlib qt
# %matplotlib notebook

%config InlineBackend.figure_format = 'retina'


# Jupytper Config
# %config Completer.use_jedi = False
# %config IPCompleter.greedy = True
# pd.set_option('display.max_columns', None)
# pd.options.display.max_rows = 30
# pd.set_option('display.max_colwidth', None)

pd.options.mode.chained_assignment = 'raise'


# Import External Files

In [9]:
# Data file produced by tests.
df = pd.read_csv('./CSV/colour_data_clustered.csv')

# Prevents CSV Tuples being imported as Strings via ast — Abstract Syntax Trees
import ast

df.ishihara_list = df.ishihara_list.apply(ast.literal_eval)
df.COLORS_ON = df.COLORS_ON.apply(ast.literal_eval)
df.COLORS_OFF = df.COLORS_OFF.apply(ast.literal_eval)

In [10]:
df.head().append(df.head(-5))

,user,correct,recorded_result,mask_image,cb_type1,cb_type2,ncb,datetime,random_spread,pallet_used,pallet_values,ishihara_list,COLORS_ON,COLORS_OFF,cluster_classification
0,unknown_1,1,5,5,0,0,1,16/6/20 11:22,10,bear,"[#e26f39,#f6c458,#e09c3e,#e69e40,#ed7b3d,#f5be...","[(220, 105, 53), (239, 192, 86), (227, 162, 61...","[(220, 105, 53), (239, 192, 86), (227, 162, 61...","[(71, 138, 38), (92, 73, 38), (153, 165, 49), ...",0
1,unknown_1,1,C,C,0,0,1,16/6/20 11:20,10,bear,"[#e26f39,#f6c458,#e09c3e,#e69e40,#ed7b3d,#f5be...","[(232, 116, 65), (247, 198, 95), (221, 152, 56...","[(232, 116, 65), (247, 198, 95), (221, 152, 56...","[(65, 149, 38), (99, 72, 41), (168, 165, 50), ...",0
2,unknown_1,1,D,D,0,0,1,16/6/20 11:30,10,bear,"[#e26f39,#f6c458,#e09c3e,#e69e40,#ed7b3d,#f5be...","[(226, 114, 54), (253, 186, 87), (233, 148, 70...","[(226, 114, 54), (253, 186, 87), (233, 148, 70...","[(75, 142, 40), (96, 64, 34), (162, 152, 41), ...",0
3,unknown_1,1,N,N,0,0,1,16/6/20 11:21,10,bear,"[#e26f39,#f6c458,#e09c3e,#e69e40,#ed7b3d,#f5be...","[(231, 116, 53), (253, 203, 81), (215, 149, 67...","[(231, 116, 53), (253, 203, 81), (215, 149, 67...","[(70, 149, 26), (96, 73, 48), (163, 158, 57), ...",0
4,unknown_1,0,N,E,0,0,1,16/6/20 11:22,10,bear,"[#e26f39,#f6c458,#e09c3e,#e69e40,#ed7b3d,#f5be...","[(223, 102, 47), (244, 188, 85), (217, 147, 55...","[(223, 102, 47), (244, 188, 85), (217, 147, 55...","[(63, 147, 29), (103, 82, 46), (155, 153, 53),...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5811,female_ncb_29,1,B,B,0,1,1,27:28.8,10,standard_3,"[#89B270, #7AA45E, #B6C674, #7AA45E, #B6C674, ...","[(146, 178, 112), (129, 168, 94), (185, 205, 1...","[(146, 178, 112), (129, 168, 94), (185, 205, 1...","[(238, 143, 43), (201, 124, 92), (226, 132, 97...",0
5812,female_ncb_29,1,2,2,0,1,1,28:01.2,10,standard_3,"[#89B270, #7AA45E, #B6C674, #7AA45E, #B6C674, ...","[(138, 172, 111), (122, 171, 101), (177, 198, ...","[(138, 172, 111), (122, 171, 101), (177, 198, ...","[(246, 153, 36), (206, 115, 102), (236, 133, 1...",0
5813,female_ncb_29,1,5,5,0,1,1,30:41.7,10,standard_3,"[#89B270, #7AA45E, #B6C674, #7AA45E, #B6C674, ...","[(137, 176, 115), (116, 158, 95), (180, 196, 1...","[(137, 176, 115), (116, 158, 95), (180, 196, 1...","[(252, 156, 41), (196, 126, 83), (240, 145, 11...",0
5814,unknown_30,1,C,C,0,1,1,21:32.3,10,standard_3,"[#89B270, #7AA45E, #B6C674, #7AA45E, #B6C674, ...","[(127, 176, 114), (119, 173, 101), (190, 196, ...","[(127, 176, 114), (119, 173, 101), (190, 196, ...","[(251, 145, 40), (203, 123, 90), (222, 145, 11...",0


### Each time a colourblind test is used a pallet is chosen.  
It's made up of 12 colours (in hex), that are then randomised a little.  
1-6 go 'on_' the symbol and 7-12 fill in the background.  

[#e26f39,#f6c458,#e09c3e,#e69e40,#ed7b3d,#f5be67,#469224,#604929,#a09f33,#dfd548,#89bcb2,#dccb47]

### RGB Values Returned

Two columns are of interest are returned from each test,   
each row con_taining a list of 6 colours with Red/Green/Blue Values  

COLORS_ON : [(220,105,53),(239,192,86),(227,162,61),(222,153,60),(235,113,53),(240,191,102)]  
COLORS_OFF : [(71,138,38),(92,73,38),(153,165,49),(221,221,70),(144,190,182),(224,196,63)]  
  
I need to isolate the colours into groups based on_ the colours.  

A single tuple is a colour, made up of (Red, Green, Blue) values.  

I need to 
[(a,b,c),(d,e,f) ...  ]  
into: ab,bc,ca, de,ef,fd ... etc  



## First: Filter the Users by response
- Take a selection of respon_ses from the entire data pool that have the 'most correct' answers for that particular set of colours.  
- The 'best' answers (most failed in most cases) are used to build an adjusted set of colours to test in the next round.

In [11]:
CALIBRATION_GROUP = pd.DataFrame() # add all selections from loop to df.

for pallet in df['pallet_used'].unique():

    df_pallet = df[df['pallet_used'] == pallet] # iterate for the pallet.
    
    # cluster_classification'] == 0 : ncb
    # cluster_classification'] == 1 : cb_type1
    # cluster_classification'] == 2 : outliers
    # cluster_classification'] == 3 : cb_type2

    # Set for type1 colourblind correct
    df1 = df_pallet[(df_pallet['cb_type1'] == 1) & (df_pallet['cluster_classification'] == 0) & (df_pallet['correct'] == 0)] 
    df2 = df_pallet[(df_pallet['cb_type1'] == 1) & (df_pallet['cluster_classification'] == 1) & (df_pallet['correct'] == 1)]
    df3 = df_pallet[(df_pallet['cb_type1'] == 1) & (df_pallet['cluster_classification'] == 2) & (df_pallet['correct'] == 0)]
    df4 = df_pallet[(df_pallet['cb_type1'] == 1) & (df_pallet['cluster_classification'] == 3) & (df_pallet['correct'] == 1)]

    # Set for type2 colourblind correct
    df5 = df_pallet[(df_pallet['cb_type2'] == 1) & (df_pallet['cluster_classification'] == 0) & (df_pallet['correct'] == 0)] 
    df6 = df_pallet[(df_pallet['cb_type2'] == 1) & (df_pallet['cluster_classification'] == 1) & (df_pallet['correct'] == 1)]
    df7 = df_pallet[(df_pallet['cb_type2'] == 1) & (df_pallet['cluster_classification'] == 2) & (df_pallet['correct'] == 0)]
    df8 = df_pallet[(df_pallet['cb_type2'] == 1) & (df_pallet['cluster_classification'] == 3) & (df_pallet['correct'] == 1)]

    # Set for ncb correct
    df9 = df_pallet[(df_pallet['ncb'] == 1) & (df_pallet['cluster_classification'] == 0) & (df_pallet['correct'] == 1)] 
    df10 = df_pallet[(df_pallet['ncb'] == 1) & (df_pallet['cluster_classification'] == 1) & (df_pallet['correct'] == 0)]
    df11 = df_pallet[(df_pallet['ncb'] == 1) & (df_pallet['cluster_classification'] == 2) & (df_pallet['correct'] == 0)]
    df12 = df_pallet[(df_pallet['ncb'] == 1) & (df_pallet['cluster_classification'] == 3) & (df_pallet['correct'] == 0)] 

    calibration = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12],axis=0) #     This pulls together the criteria for improvement.
    calibration = calibration[['pallet_used','ishihara_list']]

    CALIBRATION_GROUP = pd.concat([CALIBRATION_GROUP,calibration],axis=0)

CALIBRATION_GROUP = CALIBRATION_GROUP.reset_index()
print(f'Dataset ready for calibration')

Dataset ready for calibration


## For Each Pallet Group the values

- Now that the best values are grouped.
- The goal here is (red/green) (green/blue) (blue/red) tuples, for each colour, in each pallet, of every respon_se.
- These are eventually fed into DBSCAN clustering to shave off the outliers.
- Then we use 3D kmeans clustering, which finds a mean-center of our best batch of respon_ses.
- This becomes the base colour for our new pallet to test people with.


In [12]:
def quickplot(x,y):
    return sns.jointplot(x, y)

# ploting the 3d space.
def colourspace_plot(pallet):
    fig = plt.figure(figsize=(12,12))
    ax = fig.add_subplot(111, projection='3d')
    plt.rcParams['legend.fontsize'] = 10 

    ax.scatter(xon_red, yon_green, zon_blue, marker='x', alpha=0.7, label='rgb_on')
    ax.scatter(xoff_red, yoff_green, zoff_blue, marker='o', alpha=0.4, label='rgb_off')

    ax.set_xlabel('Red')
    ax.set_ylabel('Green')
    ax.set_zlabel('Blue')
    ax.legend(loc='lower right')

    plt.show()

### Define global vars

In [13]:
NEW_COLOURS_LIST = pd.DataFrame()
PALLET_NAMES = calibration['pallet_used'].unique()
CALIBRATION_REPORT = {}

### Calibarte Pallets.
**Warning: Computationally Expensive**

In [14]:
def create_calibration_pallet(CALIBRATION_GROUP):
    for pallet in CALIBRATION_GROUP['pallet_used'].unique(): # this cycles for each pallet used in the data individually
        df_pallet = CALIBRATION_GROUP[CALIBRATION_GROUP['pallet_used'] == pallet] # iterate for the pallet.

        if len(df_pallet['pallet_used']) > 30: # check there are enough samples
            print(f"{pallet} has {len(df_pallet['pallet_used'])} suitable responses for calibration.")
        else:
            print(f"WARNING: for {pallet} we have less than 30 samples. Skipping")
            continue

        # Stage 1: Dictionary for iteration, made up of all colours.
        colours = {
            'colour_1' : {
                'red' : [colour[0][0] for colour in df_pallet['ishihara_list']],
                'green': [colour[0][1] for colour in df_pallet['ishihara_list']],
                'blue' : [colour[0][2] for colour in df_pallet['ishihara_list']],
            },
            'colour_2' : {
                'red' : [colour[1][0] for colour in df_pallet['ishihara_list']],
                'green': [colour[1][1] for colour in df_pallet['ishihara_list']],
                'blue' : [colour[1][2] for colour in df_pallet['ishihara_list']],
            },
            'colour_3' : {
                'red' : [colour[2][0] for colour in df_pallet['ishihara_list']],
                'green': [colour[2][1] for colour in df_pallet['ishihara_list']],
                'blue' : [colour[2][2] for colour in df_pallet['ishihara_list']],
            },
            'colour_4' :{
                'red' : [colour[3][0] for colour in df_pallet['ishihara_list']],
                'green': [colour[3][1] for colour in df_pallet['ishihara_list']],
                'blue' : [colour[3][2] for colour in df_pallet['ishihara_list']],
            },
            'colour_5' :{
                'red' : [colour[4][0] for colour in df_pallet['ishihara_list']],
                'green': [colour[4][1] for colour in df_pallet['ishihara_list']],
                'blue' : [colour[4][2] for colour in df_pallet['ishihara_list']],
            },
            'colour_6' :{
                'red' : [colour[5][0] for colour in df_pallet['ishihara_list']],
                'green': [colour[5][1] for colour in df_pallet['ishihara_list']],
                'blue' : [colour[5][2] for colour in df_pallet['ishihara_list']],
            },
            'colour_7' :{
                'red' : [colour[6][0] for colour in df_pallet['ishihara_list']],
                'green': [colour[6][1] for colour in df_pallet['ishihara_list']],
                'blue' : [colour[6][2] for colour in df_pallet['ishihara_list']],
            },
            'colour_8' :{
                'red' : [colour[7][0] for colour in df_pallet['ishihara_list']],
                'green': [colour[7][1] for colour in df_pallet['ishihara_list']],
                'blue' : [colour[7][2] for colour in df_pallet['ishihara_list']],
            },
            'colour_9' :{
                'red' : [colour[8][0] for colour in df_pallet['ishihara_list']],
                'green': [colour[8][1] for colour in df_pallet['ishihara_list']],
                'blue' : [colour[8][2] for colour in df_pallet['ishihara_list']],
            },
            'colour_10' :{
                'red' : [colour[9][0] for colour in df_pallet['ishihara_list']],
                'green': [colour[9][1] for colour in df_pallet['ishihara_list']],
                'blue' : [colour[9][2] for colour in df_pallet['ishihara_list']],
            },
            'colour_11' :{
                'red' : [colour[10][0] for colour in df_pallet['ishihara_list']],
                'green': [colour[10][1] for colour in df_pallet['ishihara_list']],
                'blue' : [colour[10][2] for colour in df_pallet['ishihara_list']],
            },
            'colour_12' :{
                'red' : [colour[11][0] for colour in df_pallet['ishihara_list']],
                'green': [colour[11][1] for colour in df_pallet['ishihara_list']],
                'blue' : [colour[11][2] for colour in df_pallet['ishihara_list']],
            }
        }

        # Stage 2: Convert Dictionary to pandas DataFrame
        coloursdf = pd.DataFrame(colours)

        # Create List of Pairs For Mashing.
        calibrated_pallet = []

        pallet_report = {}

        # Stage 3: Pairing & Clustering
        for colour,rgb in coloursdf.items():
            red_green = pd.DataFrame({'red': rgb[0],'green': rgb[1]})
            green_blue = pd.DataFrame({'green': rgb[1],'blue': rgb[2]})
            blue_red = pd.DataFrame({'blue': rgb[2],'red': rgb[0]})

            calibrator = [red_green,green_blue,blue_red] # List of vectors between R,G,B  X,Y,Z points.

            # Flexible DBSCAN Variables.
            eps_val = 15 # EPS Max dist is 20, for spread 10. 10 = 50% of range
            min_samples_val = int(len(calibrator[0].iloc[:, 0]) * .75) # 90% of the total pool required to make 1 cluster.

            # Stage 4: DBSCAN
            cluster_centers = [] # list of pair centers.
            for pair in calibrator:
                status = {}
                # prune outliers
                status['primary_count'] = len(pair.iloc[:, 0]) # Get data len # consider using .shape[0]
                pair = pair[(np.abs(stats.zscore(pair.iloc[:, 0])) < 3)] # uses the zscore function of df less than a SD of 3
                pair = pair[(np.abs(stats.zscore(pair.iloc[:, 1])) < 3)] # uses the zscore function of df less than a SD of 3
                status['secondary_count'] = len(pair.iloc[:, 0]) # Get data len
                status['primary_removed'] = status['primary_count'] - status['secondary_count'] # Get dif  in data len

                dbscan = DBSCAN(eps=eps_val, min_samples=min_samples_val, metric = 'euclidean').fit_predict(pair) # create a big cluster
                pair['dbscan'] = dbscan # assign it as an array
                dbscan = pair[pair['dbscan'] == 0] # Take cluster 0 
                dbscan = dbscan.drop('dbscan',axis=1)
                status['third_count'] = len(dbscan.iloc[:, 0]) # Get data len
                status['secondary_removed'] = status['secondary_count'] - status['third_count'] # Get dif  in data len

                # KMeans, finding new cluster centers.
                kmeans = KMeans(n_clusters=1).fit(dbscan) # fit KMEANS
                cluster_centers.append(kmeans.cluster_centers_) # Take the center value of the cluster.

    #                 quickplot(dbscan)

                pallet_report[str(pair)] = status
            CALIBRATION_REPORT[pallet] = pallet_report

            red_corrected = int((cluster_centers[0][0][0] + cluster_centers[2][0][1]) / 2) # red
            green_corrected = int((cluster_centers[0][0][1] + cluster_centers[1][0][0]) / 2) # green
            blue_corrected = int((cluster_centers[1][0][1] + cluster_centers[2][0][0]) / 2) # blue

            calibrated_rgb=[red_corrected,green_corrected,blue_corrected]
            calibrated_rgb = '#%02x%02x%02x' % (red_corrected, green_corrected, blue_corrected) # Turn Tuple to Hex
            calibrated_pallet.append(calibrated_rgb) # add the list of colours to a pallet list

        NEW_COLOURS_LIST[pallet] = calibrated_pallet
    

    print('All pallets have been calibrated.')

In [15]:
create_calibration_pallet(CALIBRATION_GROUP)

bear has 318 suitable responses for calibration.
bluey2 has 266 suitable responses for calibration.
camo2 has 485 suitable responses for calibration.
greys has 457 suitable responses for calibration.
inverse_ring has 331 suitable responses for calibration.
orn_grn has 470 suitable responses for calibration.
pinks has 401 suitable responses for calibration.
reversed_ring has 394 suitable responses for calibration.
ring has 654 suitable responses for calibration.
standard_1 has 283 suitable responses for calibration.
standard_2 has 281 suitable responses for calibration.
standard_3 has 454 suitable responses for calibration.
All pallets have been calibrated.


In [16]:
NEW_COLOURS_LIST.T

,0,1,2,3,4,5,6,7,8,9,10,11
bear,#e16f38,#f4c257,#e09b3d,#e59d3e,#ec7b3b,#f3bd66,#459123,#5f4829,#9f9d32,#ded447,#88bbb1,#dbcb46
bluey2,#3180bd,#07519b,#3182bd,#08519b,#3181bd,#08519b,#71219e,#995abe,#71219e,#995abe,#71219f,#995abe
camo2,#f86949,#dd2c26,#f86949,#dd2c26,#f86949,#dd2c26,#8f6c17,#c88c41,#8e6c17,#c88c41,#8f6c17,#c88c41
greys,#f1bacd,#a33a66,#cd7896,#f1bacd,#a33a66,#cd7896,#959595,#cbcbcb,#626262,#959595,#cbcbcb,#626262
inverse_ring,#394a77,#b1707f,#944f7a,#505166,#303f6f,#bc4e74,#ca5f7e,#b37e93,#d9557d,#b45c79,#656076,#53506e
orn_grn,#a8a137,#bcb540,#9e9405,#a8a137,#bcb540,#9e9404,#78bf74,#10a004,#4fb154,#78bf74,#10a004,#50b154
pinks,#f92d7b,#f92d7b,#f92e7b,#f92d7b,#f92d7b,#f92d77,#fa3ac9,#f931a8,#f92d7f,#f92d67,#f92e5a,#f92e54
reversed_ring,#507433,#9c6d81,#776b78,#71877b,#5b6c2b,#5a6dc1,#476fe2,#437553,#716f69,#694e6e,#52364f,#666d90
ring,#2f8962,#a88881,#8c6750,#7d8474,#1d804c,#bb6559,#cf704a,#8b6464,#df9363,#ac7b4f,#6f7549,#68833d
standard_1,#f7b981,#eaa06f,#f8cc83,#f7ba81,#eaa070,#f8cc83,#9ba493,#abb3a4,#bab863,#d6d9a8,#e4d47c,#d0d5af


In [17]:
# These values will ultimately replace the existing values below. (Note: Different dicts. at the moment.)
pallet_dict = pd.read_csv('./CSV/pallets_dictionary_new.csv')
pallet_dict

,pallet_name,hex0,hex1,hex2,hex3,hex4,hex5,hex6,hex7,hex8,hex9,hex10,hex11,cb_type1,cb_type2,ncb
0,Plates_2-5,#ea996d,#f87562,#e9cfc5,#f4542d,#fd8c42,#f87570,#c6a04d,#555531,#827746,#898144,#c5aa3b,#dddddd,0,0,1
1,Plates_6-9,#7d8762,#bcc967,#69603a,#baa955,#ad9556,#bcc966,#d65153,#fc8b67,#fe9466,#e85a41,#f0b55e,#fe733e,0,0,1
2,Plates_10-13,#bc5325,#f5a957,#e87941,#ff793b,#c74e25,#f9703b,#f4d66f,#bcb778,#796b3b,#b39e47,#5c7579,#796b3b,0,0,1
3,Plates_14-17,#a3ae5e,#4c5b35,#91ae46,#616a3b,#a0b164,#a5ae5f,#ee5149,#f39935,#fad600,#fb9a5e,#f3d92d,#eb952e,0,0,1
4,Plates_18-21,#f7422a,#f7682e,#f87a58,#f56031,#e36f2f,#f87a59,#7e8149,#a86d45,#634330,#806f56,#375e42,#af7648,0,0,1
5,Plates_22-27P,#fe5b55,#f95c50,#9d4053,#f55d52,#722F3F,#fe5b51,#7d6b5f,#23262d,#4e4245,#856C55,#827059,#705F5C,0,1,1
6,Plates_22-27D,#f44d3b,#E9453D,#efa892,#fb5c51,#E77262,#efa893,#7d6b5f,#23262d,#4e4245,#856C55,#827059,#705F5C,1,0,1
7,Plates_30-31,#9eab5e,#5c6746,#f6d01e,#bdbe75,#6a6f4e,#9caf5d,#f6d01e,#f9654c,#faa06d,#d5412c,#facc33,#f87653,0,0,1
8,Plates_32-33,#f98f4f,#dc522e,#f2693c,#eea055,#ff6839,#eea050,#6f6a22,#d0bd5a,#8b8134,#6b8382,#6c7f82,#d6b04a,0,0,1
9,Plates_34-35,#a6ad62,#576f54,#999d62,#69653e,#bfaf5f,#a38f4f,#fac243,#d95415,#ff814f,#fdb654,#b44747,#bd7270,0,0,1


## Once enough data is collected, replace the existing dictionary values with the improved values.

Fin.